# **Problem: Residual Neural Network in tensorflow. Train on cats and dogs dataset.**

Python program to train cats and dogs dataset using Residual Neural Network in tensorflow (without transfer learning).

Run all the cells. After executing the last cell, you will get the accuracy of the trained model.

**Notes:**

Following things are needed to be checked before running the program.
  1. Keras module is needed to be installed.
  2. gdown module is needed to be installed.
  3. Check whether you have given the correct location of your dataset file.
  4. You should have access to the file in the Google Drive.
  

# **Import Modules**

In [1]:
# Import Resnet50 model
from keras.applications.resnet import ResNet50

# Import ImageDataGenerator to generate train and test batches
from keras.preprocessing.image import ImageDataGenerator 

# Import Model to create the final model
from keras.models import Model

# Import keras layers
from keras.layers import Dense,GlobalAveragePooling2D

# Import adam optimizer
from tensorflow.keras.optimizers import Adam

# Import preprocess_input for ImageDataGenerator class
from keras.applications.resnet_v2 import preprocess_input

# Import gdown module to download files from google drive
import gdown

# Import zip file module to open the zip file
from zipfile import ZipFile


## **Get the file location from google drive and download** **bold text**

In [ ]:
# Please change the URL as needed (make sure you have the access to the file)

url = 'https://drive.google.com/file/d/1fMHrqIY0QYEj9qFUFsDuF949Jo-UWzVX/view?usp=sharing'

# Derive the file id from the URL
file_id = url.split('/')[-2]

# Derive the download url of the the file
download_url = 'https://drive.google.com/uc?id=' + file_id

# Give the location you want to save it in your local machine
file_location = 'cats_and_dogs.zip'

# Download the file from drive to your local machine
gdown.download(download_url, file_location, quiet=False)

# **Unzip the zip dataset**

In [ ]:
!unzip /content/cats_and_dogs.zip -d "/content/unzipped_folder/"

# **Start the prediction operation**

In [ ]:
# Give image size and shape
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Define the number of classes
num_classes = 2

# Define train path and test path
train_path = '/content/unzipped_folder/training_set/training_set'
test_path = '/content/unzipped_folder/test_set/test_set'

# Go through the train directory to obtain cateogries
train_batches = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
                train_path ,target_size=(IMG_SIZE,IMG_SIZE),batch_size=24,class_mode='categorical')

# Go through the test directory to obtain cateogries
test_batches = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
                test_path ,target_size=(IMG_SIZE,IMG_SIZE),batch_size=24,class_mode='categorical')

# Derive the resnet pretrained model without weights
base_model = ResNet50(weights= None, include_top=False, input_shape=IMG_SHAPE)

# Get the last layer and add a few extra layers to it
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a layer with softmax activation
predictions = Dense(num_classes, activation= 'softmax')(x)

# Get the final model
model = Model(inputs = base_model.input, outputs = predictions)

# Define the base learning rate
base_learning_rate = 0.0001

# Use Adam optimizer as the optimizer of the model
adam = Adam(lr=base_learning_rate)

# Loss function IS categorical cross entropy
# Accuracy is the evaluation metric
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the training data
model.fit(train_batches, steps_per_epoch=2, epochs=2, validation_data=test_batches)

# Evaluate the model
preds = model.evaluate(test_batches, steps = 20)

# Print the accuracy of the model
print ("Accuracy = " + str(preds[1]))